In [ ]:
from lxml import html
import requests

Get list of all datasets from PO.DAAC at JPL using the max items of 400 per request and running until we get a response containing less than 400 entries

In [ ]:
iteration = 0
items_per_iteration = 400
entry_list = []
while True:
    response = requests.get("https://podaac.jpl.nasa.gov/ws/search/dataset/?startIndex=" + str(iteration * items_per_iteration) + "&itemsPerPage=" + str(items_per_iteration) + "&format=html")
    html_tree = html.fromstring(response.content)
    entries = html_tree.cssselect("div.entry")
    entry_list += entries
    if len(entries) < items_per_iteration:
        break
    iteration += 1

Creates a list of datasets and their storage location

In [ ]:
ftps = []
unknowns = []
for entry in entry_list:
    found = False
    links = entry.cssselect("div.links")
    for link in links:
        for child in link.getchildren():
            if "FTP URL" in child.text:
                found = True
                ftps.append({"dataset" : entry.getchildren()[0].text, "url" : child.attrib['href']})
    if not found:
        unknowns.append(entry.getchildren()[0].text)

In [ ]:
unknowns

In [ ]:
ftps

Go to each url and crawl through the data for files (Still working on this)

In [ ]:
from ftplib import FTP
from urllib.parse import urlparse

In [ ]:
parsed = urlparse(ftps[0]['url'])

In [ ]:
ftp = FTP(parsed.netloc)
ftp.login()

In [ ]:
def traverse_ftp_directory(ftp, directory):
    current_directory = ftp.pwd()
    files = []
    try:
        #If directory is a directory change to it otherwise throws an Exception because it is a file
        ftp.cwd(directory)
        items = ftp.nlst()
        for item in items:
            files += traverse_ftp_directory(ftp, directory + "/" + item)
        return files
    except Exception:
        return [directory]

In [ ]:
traverse_ftp_directory(ftp, "/allData/ghrsst/data/L2P/TMI/REMSS")

Testing some ideas

In [ ]:
files = []
try:
    ftp.cwd("/allData/ghrsst/data/L2P/TMI/REMSS/")
    items = ftp.nlst()
    for item in items:
        files += ["/allData/ghrsst/data/L2P/TMI/REMSS" + "/" + item]
except Exception:
    print("Done")

In [ ]:
files